## Categorizing US counties

In [49]:
import pandas as pd

### Import County-level Covid Data
Source: NY Times (https://github.com/nytimes/covid-19-data)

In [50]:
url = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"

covid_by_counties = pd.read_csv(url, error_bad_lines=False)

print(covid_by_counties.shape)
covid_by_counties.head()

(1212589, 6)


date     county       state     fips  cases  deaths
0  2020-01-21  Snohomish  Washington  53061.0      1     0.0
1  2020-01-22  Snohomish  Washington  53061.0      1     0.0
2  2020-01-23  Snohomish  Washington  53061.0      1     0.0
3  2020-01-24       Cook    Illinois  17031.0      1     0.0
4  2020-01-24  Snohomish  Washington  53061.0      1     0.0

In [51]:
covid_by_counties = covid_by_counties[covid_by_counties["state"] != "Guam"]
covid_by_counties = covid_by_counties[covid_by_counties["state"] != "Virgin Islands"]
covid_by_counties = covid_by_counties[covid_by_counties["state"] != "Puerto Rico"]
covid_by_counties = covid_by_counties[covid_by_counties["state"] != "Northern Mariana Islands"]

In [52]:
covid_by_counties[covid_by_counties["fips"].isnull()].groupby("county").sum()

fips      cases     deaths
county                                   
Joplin          0.0     828153    15496.0
Kansas City     0.0    6369699    80066.0
New York City   0.0  134845544  8859621.0
Unknown         0.0   12053242   263734.0

In [53]:
covid_by_counties.loc[covid_by_counties["county"] == "New York City", "fips"] = 36061
covid_by_counties.loc[covid_by_counties["county"] == "Joplin", "fips"] = 29097
covid_by_counties.loc[covid_by_counties["county"] == "Kansas City", "fips"] = 29095

covid_by_counties[covid_by_counties["fips"].isnull()].groupby("county").sum()
covid_by_counties = covid_by_counties[covid_by_counties["fips"].isnull() == False]

In [54]:
before_classes = covid_by_counties[covid_by_counties["date"] == "2020-08-01"]

In [55]:
after_classes = covid_by_counties[covid_by_counties["date"] == "2020-10-01"]

### Import county category data
Source: NCHS Urban-Rural Classification Scheme for Counties

##### The NCHS has developed a six-level urban-rural classification scheme for U.S. counties and county-equivalent entities.
1. Metropolitan counties: Large central metro counties in MSA of 1 million population that: 1) contain the entire population of the largest principal city of the MSA, or 2) are completely contained within the largest principal city of the MSA, or 3) contain at least 250,000 residents of any principal city in the MSA.
2. Large fringe metro counties in MSA of 1 million or more population that do not qualify as large central 
3. Medium metro counties in MSA of 250,000-999,999 population.
4. Small metro counties are counties in MSAs of less than 250,000 population.
5. Micropolitan counties in micropolitan statistical area
6. Noncore counties not in micropolitan statistical areas
    
##### We can farther collapse these six categories into urban, suburban, and rural:
- Urban: 1 (Large central metro)
- Suburban: 2, 3, 4 (Large fringe metro counties, medium metro, and small metro counties)
- Rural: 5 and 6 (Micropolitan counties and Noncore counties)
    

In [56]:
county_categories = pd.read_excel("NCHSURCodes2013.xlsx")
print(county_categories.shape)
county_categories.head()

(3149, 9)


FIPS code State Abr.     County name                 CBSA title  \
0       1001         AL  Autauga County             Montgomery, AL   
1       1003         AL  Baldwin County  Daphne-Fairhope-Foley, AL   
2       1005         AL  Barbour County                        NaN   
3       1007         AL     Bibb County      Birmingham-Hoover, AL   
4       1009         AL   Blount County      Birmingham-Hoover, AL   

  CBSA 2012 pop County 2012 pop  2013 code  2006 code 1990-based code  
0        377149           55514          3          3               3  
1        190790          190790          4          5               3  
2             .           27201          6          5               5  
3       1136650           22597          2          2               6  
4       1136650           57826          2          2               3

In [57]:
selected = county_categories[["FIPS code", "2013 code"]]

### Merging and Fixing

The first merge attempt shows that three of the counties in this dataset do not come with fips county code that identifies them. They are New York City, Kansas City, and Joplin. 

We will fill in the fips code for New York City with that of New York County because they generally refer to the same region. 

Joplin (Missouri) is a city in both Jasper and Newton County. Since both are designated by the NCHS system as small metro counties and Joplin is mostly in Jasper County, we will fill in the Joplin FIPS code with the Jasper County FIPS code.

Kansas City (Missouri) straddles the Kansas-Missouri state line. The bulk of it is in Jackson County, Missouri, but parts of it lie in Clay, Cass, and Platte Counties. We will fill in the Kansas City FIPS with that of Jackson County.

In [58]:
with_categories_left_before = pd.merge(before_classes, selected, left_on = "fips", right_on = "FIPS code", how = "left")
with_categories_left_before.head(10)

date    county    state    fips  cases  deaths  FIPS code  2013 code
0  2020-08-01   Autauga  Alabama  1001.0   1030    21.0     1001.0        3.0
1  2020-08-01   Baldwin  Alabama  1003.0   3142    22.0     1003.0        4.0
2  2020-08-01   Barbour  Alabama  1005.0    602     5.0     1005.0        6.0
3  2020-08-01      Bibb  Alabama  1007.0    368     2.0     1007.0        2.0
4  2020-08-01    Blount  Alabama  1009.0    792     3.0     1009.0        2.0
5  2020-08-01   Bullock  Alabama  1011.0    443    12.0     1011.0        6.0
6  2020-08-01    Butler  Alabama  1013.0    757    36.0     1013.0        6.0
7  2020-08-01   Calhoun  Alabama  1015.0   1480     9.0     1015.0        4.0
8  2020-08-01  Chambers  Alabama  1017.0    861    38.0     1017.0        5.0
9  2020-08-01  Cherokee  Alabama  1019.0    243     8.0     1019.0        6.0

In [59]:
with_categories_left_after = pd.merge(after_classes, selected, left_on = "fips", right_on = "FIPS code", how = "left")
with_categories_left_after.head(10)

date    county    state    fips  cases  deaths  FIPS code  2013 code
0  2020-10-01   Autauga  Alabama  1001.0   1798    28.0     1001.0        3.0
1  2020-10-01   Baldwin  Alabama  1003.0   5997    53.0     1003.0        4.0
2  2020-10-01   Barbour  Alabama  1005.0    898     7.0     1005.0        6.0
3  2020-10-01      Bibb  Alabama  1007.0    672    10.0     1007.0        2.0
4  2020-10-01    Blount  Alabama  1009.0   1634    15.0     1009.0        2.0
5  2020-10-01   Bullock  Alabama  1011.0    612    16.0     1011.0        6.0
6  2020-10-01    Butler  Alabama  1013.0    919    40.0     1013.0        6.0
7  2020-10-01   Calhoun  Alabama  1015.0   3587    44.0     1015.0        4.0
8  2020-10-01  Chambers  Alabama  1017.0   1181    42.0     1017.0        5.0
9  2020-10-01  Cherokee  Alabama  1019.0    624    13.0     1019.0        6.0

### Import US Population Data at the County Level

Source: https://www.ers.usda.gov/data-products/county-level-data-sets/download-data/



In [60]:
population = pd.read_excel("PopulationEstimates.xls")
print(population.shape)
population.head()

(3273, 165)


FIPStxt State       Area_Name  Rural-urban_Continuum Code_2003  \
0        0    US   United States                              NaN   
1     1000    AL         Alabama                              NaN   
2     1001    AL  Autauga County                              2.0   
3     1003    AL  Baldwin County                              4.0   
4     1005    AL  Barbour County                              6.0   

   Rural-urban_Continuum Code_2013  Urban_Influence_Code_2003  \
0                              NaN                        NaN   
1                              NaN                        NaN   
2                              2.0                        2.0   
3                              3.0                        5.0   
4                              6.0                        6.0   

   Urban_Influence_Code_2013  Economic_typology_2015  CENSUS_2010_POP  \
0                        NaN                     NaN        308745538   
1                        NaN                     NaN          4779736   
2                        2.0                     0.0            54571   
3                        2.0                     5.0           182265   
4                        6.0                     3.0            27457   

   ESTIMATES_BASE_2010  ...  R_DOMESTIC_MIG_2019  R_NET_MIG_2011  \
0            308758105  ...                  NaN             NaN   
1              4780125  ...             1.917501        0.578434   
2                54597  ...             4.847310        6.018182   
3               182265  ...            24.017829       16.641870   
4                27455  ...            -5.690302        0.292676   

   R_NET_MIG_2012  R_NET_MIG_2013  R_NET_MIG_2014  R_NET_MIG_2015  \
0             NaN             NaN             NaN             NaN   
1        1.186314        1.522549        0.563489        0.626357   
2       -6.226119       -3.902226        1.970443       -1.712875   
3       17.488579       22.751474       20.184334       17.725964   
4       -6.897817       -8.132185       -5.140431      -15.724575   

   R_NET_MIG_2016  R_NET_MIG_2017  R_NET_MIG_2018  R_NET_MIG_2019  
0             NaN             NaN             NaN             NaN  
1        0.745172        1.090366        1.773786        2.483744  
2        4.777171        0.849656        0.540916        4.560062  
3       21.279291       22.398256       24.727215       24.380567  
4      -18.238016      -24.998528       -8.754922       -5.165664  

[5 rows x 165 columns]

In [61]:
population = population[population["State"] != "PR"]
population = population[population["State"] != "US"]
population = population[population["Rural-urban_Continuum Code_2013"].isnull() == False]

In [62]:
pop_select = population[["FIPStxt", "POP_ESTIMATE_2019"]]

In [63]:
pop_merge_before = pd.merge(with_categories_left_before, pop_select, left_on = "fips", right_on = "FIPStxt", how = "left")

In [64]:
pop_merge_after = pd.merge(with_categories_left_after, pop_select, left_on = "fips", right_on = "FIPStxt", how = "left")

### Dividing into Urban, Suburban, and Rural

In [65]:
pop_merge_before.loc[pop_merge_before["2013 code"] == 1, "category"] = "Urban"
pop_merge_before.loc[pop_merge_before["2013 code"] == 2, "category"] = "Suburban"
pop_merge_before.loc[pop_merge_before["2013 code"] == 3, "category"] = "Suburban"
pop_merge_before.loc[pop_merge_before["2013 code"] == 4, "category"] = "Suburban"
pop_merge_before.loc[pop_merge_before["2013 code"] == 5, "category"] = "Rural"
pop_merge_before.loc[pop_merge_before["2013 code"] == 6, "category"] = "Rural"

In [66]:
pop_merge_after.loc[pop_merge_after["2013 code"] == 1, "category"] = "Urban"
pop_merge_after.loc[pop_merge_after["2013 code"] == 2, "category"] = "Suburban"
pop_merge_after.loc[pop_merge_after["2013 code"] == 3, "category"] = "Suburban"
pop_merge_after.loc[pop_merge_after["2013 code"] == 4, "category"] = "Suburban"
pop_merge_after.loc[pop_merge_after["2013 code"] == 5, "category"] = "Rural"
pop_merge_after.loc[pop_merge_after["2013 code"] == 6, "category"] = "Rural"

In [67]:
pop_merge_before = pop_merge_before.drop(["FIPStxt"], axis = 1)
pop_merge_after = pop_merge_after.drop(["FIPStxt"], axis = 1)

In [69]:
# fips will be string version of fips code with zero padding in front.
no_na_b = pop_merge_before[pop_merge_before["fips"].isnull() == False]
no_na_b = no_na_b.astype({"fips": "float64"})
no_na_b = no_na_b.astype({"fips": "int64"})
no_na_b = no_na_b.astype({"fips": "string"})
no_na_b["fips"] = no_na_b["fips"].str.zfill(5)

no_na_b.head()

date   county    state   fips  cases  deaths  FIPS code  2013 code  \
0  2020-08-01  Autauga  Alabama  01001   1030    21.0     1001.0        3.0   
1  2020-08-01  Baldwin  Alabama  01003   3142    22.0     1003.0        4.0   
2  2020-08-01  Barbour  Alabama  01005    602     5.0     1005.0        6.0   
3  2020-08-01     Bibb  Alabama  01007    368     2.0     1007.0        2.0   
4  2020-08-01   Blount  Alabama  01009    792     3.0     1009.0        2.0   

   POP_ESTIMATE_2019  category  
0            55869.0  Suburban  
1           223234.0  Suburban  
2            24686.0     Rural  
3            22394.0  Suburban  
4            57826.0  Suburban

In [70]:
# fips will be string version of fips code with zero padding in front.
no_na_a = pop_merge_after[pop_merge_after["fips"].isnull() == False]
no_na_a = no_na_a.astype({"fips": "float64"})
no_na_a = no_na_a.astype({"fips": "int64"})
no_na_a = no_na_a.astype({"fips": "string"})
no_na_a["fips"] = no_na_a["fips"].str.zfill(5)

no_na_a.head()

date   county    state   fips  cases  deaths  FIPS code  2013 code  \
0  2020-10-01  Autauga  Alabama  01001   1798    28.0     1001.0        3.0   
1  2020-10-01  Baldwin  Alabama  01003   5997    53.0     1003.0        4.0   
2  2020-10-01  Barbour  Alabama  01005    898     7.0     1005.0        6.0   
3  2020-10-01     Bibb  Alabama  01007    672    10.0     1007.0        2.0   
4  2020-10-01   Blount  Alabama  01009   1634    15.0     1009.0        2.0   

   POP_ESTIMATE_2019  category  
0            55869.0  Suburban  
1           223234.0  Suburban  
2            24686.0     Rural  
3            22394.0  Suburban  
4            57826.0  Suburban

In [71]:
no_na_b["percent_covid"] = no_na_b["cases"]/no_na_b["POP_ESTIMATE_2019"]
no_na_b.head()

date   county    state   fips  cases  deaths  FIPS code  2013 code  \
0  2020-08-01  Autauga  Alabama  01001   1030    21.0     1001.0        3.0   
1  2020-08-01  Baldwin  Alabama  01003   3142    22.0     1003.0        4.0   
2  2020-08-01  Barbour  Alabama  01005    602     5.0     1005.0        6.0   
3  2020-08-01     Bibb  Alabama  01007    368     2.0     1007.0        2.0   
4  2020-08-01   Blount  Alabama  01009    792     3.0     1009.0        2.0   

   POP_ESTIMATE_2019  category  percent_covid  
0            55869.0  Suburban       0.018436  
1           223234.0  Suburban       0.014075  
2            24686.0     Rural       0.024386  
3            22394.0  Suburban       0.016433  
4            57826.0  Suburban       0.013696

In [72]:
no_na_a["percent_covid"] = no_na_a["cases"]/no_na_a["POP_ESTIMATE_2019"]
no_na_a.head()

date   county    state   fips  cases  deaths  FIPS code  2013 code  \
0  2020-10-01  Autauga  Alabama  01001   1798    28.0     1001.0        3.0   
1  2020-10-01  Baldwin  Alabama  01003   5997    53.0     1003.0        4.0   
2  2020-10-01  Barbour  Alabama  01005    898     7.0     1005.0        6.0   
3  2020-10-01     Bibb  Alabama  01007    672    10.0     1007.0        2.0   
4  2020-10-01   Blount  Alabama  01009   1634    15.0     1009.0        2.0   

   POP_ESTIMATE_2019  category  percent_covid  
0            55869.0  Suburban       0.032182  
1           223234.0  Suburban       0.026864  
2            24686.0     Rural       0.036377  
3            22394.0  Suburban       0.030008  
4            57826.0  Suburban       0.028257

In [80]:
comp = pd.merge(no_na_a, no_na_b[["fips", "cases", "deaths", "percent_covid"]], on = "fips")
comp = comp.rename(columns={"cases_x": "cases_10", "deaths_x": "deaths_10", "cases_y": "cases_8", "deaths_y": "deaths_8", "percent_covid_x": "percent_covid_10", "percent_covid_y": "percent_covid_8"})
comp = comp.drop(["date"], axis = 1)
comp["case diff"] = comp["cases_10"] - comp["cases_8"]
comp["death diff"] = comp["deaths_10"] - comp["deaths_8"]
comp["percent diff"] = comp["percent_covid_10"] - comp["percent_covid_8"]
comp.head()

county    state   fips  cases_10  deaths_10  FIPS code  2013 code  \
0  Autauga  Alabama  01001      1798       28.0     1001.0        3.0   
1  Baldwin  Alabama  01003      5997       53.0     1003.0        4.0   
2  Barbour  Alabama  01005       898        7.0     1005.0        6.0   
3     Bibb  Alabama  01007       672       10.0     1007.0        2.0   
4   Blount  Alabama  01009      1634       15.0     1009.0        2.0   

   POP_ESTIMATE_2019  category  percent_covid_10  cases_8  deaths_8  \
0            55869.0  Suburban          0.032182     1030      21.0   
1           223234.0  Suburban          0.026864     3142      22.0   
2            24686.0     Rural          0.036377      602       5.0   
3            22394.0  Suburban          0.030008      368       2.0   
4            57826.0  Suburban          0.028257      792       3.0   

   percent_covid_8  case diff  death diff  percent diff  
0         0.018436        768         7.0      0.013746  
1         0.014075       2855        31.0      0.012789  
2         0.024386        296         2.0      0.011991  
3         0.016433        304         8.0      0.013575  
4         0.013696        842        12.0      0.014561